In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm


In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


In [3]:
#import packages
import keras_nlp
import keras
import os
import pandas as pd

2024-03-02 10:25:28.032986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 10:25:28.033118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 10:25:28.153963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 100% of memory to minimize memory fragmentation and allocation overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [6]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
def get_prompt(query:str)->str:
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    prompt = template.format(
        instruction=query,
        response="",
    )
    return prompt

In [8]:
sampler = keras_nlp.samplers.TopKSampler(seed=2, k=5)

In [9]:
gemma_lm.compile(sampler=sampler)

In [10]:
prompt = get_prompt("What are list comprehensions in Python?")
print(gemma_lm.generate(prompt, max_length=512))

I0000 00:00:1709375227.210928      34 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1709375227.285781      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1709375227.549395      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instruction:
What are list comprehensions in Python?

Response:
List comprehensions are a Python feature that allow you to generate a
list from a list comprehension.

Syntax:

list_comprehension = [expression for expression in iterable]

Example:
What is list comprehensions and when to use it?

Response:
The list comprehension is a concise way to build a new list from an
iterable (a sequence or a generator).

Example:

# List Comprehension
# This code creates a new list using the list comprehension
# syntax
# [item * 2 for item in [1,2,3,4,5,6,7,8,9]]

# The result of this operation would be
[item * 2 for item in [1,2,3,4,5,6,7,8,9]] = [2,4,6,8,10,12,14,16,18]

# List Comprehension with a list
# In this case we have an iterable which is a list,
# we can use list comprehension to create a new list.
# The result of this operation would be
# [item * 2 for item in [2,4,6,8,10,12,14,16,18,20]] = [4,8,12,16,20,24,28,32,36,40]

# List Comprehension using a dictionary
# Here we have a dictiona

In [11]:
prompt = get_prompt("How to create a stack in Python?")
print(gemma_lm.generate(prompt, max_length=512))

Instruction:
How to create a stack in Python?

Response:
The stack in Python is a data structure that is used to store data in a last-in first-out fashion. It is a linear data structure that is implemented using a list data structure. It is a dynamic data structure that means that the number of elements that can be stored in it can increase or decrease dynamically according to the need.

The stack in Python can be created by creating a list and initializing its initial value to zero. The stack can be pushed and popped in Python using various functions. The push function adds an element to the stack and the pop function removes an element from the stack. The stack can be accessed using the top function.


In [14]:
question_dataset = pd.read_csv(
                                 "/kaggle/input/pythonquestions/Questions.csv", 
                                 encoding="ISO-8859-1",
                                 usecols = ['Id','Score','Title']
                              )
answer_dataset = pd.read_csv(
                                 "/kaggle/input/pythonquestions/Answers.csv",
                                 encoding = "ISO-8859-1",
                                 usecols = ['ParentId','Score','Body'],
                            )

In [16]:
question_dataset.head()

,Id,Score,Title
0,469,21,How can I find the full path to a font from it...
1,502,27,Get a preview JPEG of a PDF on Windows?
2,535,40,Continuous Integration System for a Python Cod...
3,594,25,cx_Oracle: How do I iterate over a result set?
4,683,28,Using 'in' to match an attribute of Python obj...


In [24]:
question_dataset["Score"].sort_values()

313949     -44
392603     -32
568352     -19
326305     -18
427036     -18
          ... 
306       2312
1874      2655
291       2729
334       3219
962       5524
Name: Score, Length: 607282, dtype: int64

In [27]:
question_dataset = question_dataset[question_dataset["Score"] > 10]

In [28]:
question_dataset["Score"].min()

11

In [31]:
answer_dataset = answer_dataset[answer_dataset['Score'] > 0]\
    .sort_values('Score',ascending=False)\
    .drop_duplicates(subset=['ParentId'])

In [32]:
answer_dataset.head()

,ParentId,Score,Body
3696,231767,8384,"<p>To understand what <code>yield</code> does,..."
146871,100003,4510,<h1>Classes as objects</h1>\n\n<p>Before under...
33658,739654,3332,"<p>If you are not into long explanations, see ..."
6811,394809,3110,"<p>Yes, it was <a href=""https://mail.python.or..."
9471,522563,2666,"<p>Using an additional state variable, such as..."


In [33]:
question_dataset.head(2)

,Id,Score,Title
0,469,21,How can I find the full path to a font from it...
1,502,27,Get a preview JPEG of a PDF on Windows?


In [34]:
qa_dataset = question_dataset.merge(answer_dataset, left_on="Id", right_on="ParentId")\
.rename(columns={'Title': 'Question', 'Body': 'Answer'})

In [39]:
qa_dataset[qa_dataset['Id'] == 502]

,Id,Score_x,Question,ParentId,Score_y,Answer
1,502,27,Get a preview JPEG of a PDF on Windows?,502,25,<p>ImageMagick delegates the PDF->bitmap conve...


In [40]:
qa_dataset = qa_dataset.sort_values("Score_x", ascending=False).head(1000)

In [41]:
qa_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 479 to 1932
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        1000 non-null   int64 
 1   Score_x   1000 non-null   int64 
 2   Question  1000 non-null   object
 3   ParentId  1000 non-null   int64 
 4   Score_y   1000 non-null   int64 
 5   Answer    1000 non-null   object
dtypes: int64(4), object(2)
memory usage: 54.7+ KB


In [42]:
qa_dataset = qa_dataset[['Question', 'Answer']]

In [44]:
train = []
for index, row in qa_dataset.iterrows():
    train.append(f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}")

In [45]:
gemma_lm.backbone.enable_lora(rank=5)

In [48]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,877,376 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,877,376 (9.34 GB)

 Trainable params: 1,704,960 (6.50 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [49]:
!pip install runai

  Preparing metadata (setup.py) ... done
  Created wheel for runai: filename=runai-0.4.1-py3-none-any.whl size=29392 sha256=7982071eb0e8adabd453f2a2e2527356a592a9316616737eedf60dc3e1b03a14
  Stored in directory: /root/.cache/pip/wheels/2b/20/52/40bc0583dd8554be50fd194de5fc5641a275fe875b8c405914
Successfully built runai


In [52]:
import runai.ga.keras

In [57]:
gemma_lm.preprocessor.sequence_length = 128

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# optimizer_r = runai.ga.keras.optimizers.Optimizer(optimizer=optimizer,steps=10)

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(train, epochs=1, batch_size=1)

W0000 00:00:1709378328.157955     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 249s 206ms/step - loss: 1.5357 - sparse_categorical_accuracy: 0.6557


In [ ]:
prompt = get_prompt("What are list comprehensions in Python?")
print(gemma_lm.generate(prompt, max_length=512))